## Загрузка бибилиотек

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### Загрузка и подготовка данных

Посмотрим на примере данных по оттоку клиентов из телеком-компании.

    Важно: XGBoost поддерживает только числовые признаки. Поэтому текстовые признаки требуют предобработки

Для обучения с Xgboost данные должны быть представлены в виде объекта DMatrix, предназначенного для представления разреженных данных. Его можно инициализировать через:

- текстовый формат libsvm,
- Numpy 2D array (чаще всего)
- бинарный буфер-файл XGBoost

Загрузим данные и осуществим минимальную предобработку.


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/OtusTeam/Bigdata-ML/master/module-8/34-xgboost/telecom_churn.csv")

In [4]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


Штаты просто занумеруем (хотя можно и лучше поступить), а признаки International plan (наличие международного роуминга), Voice mail plan (наличие голосовой почтыы) и целевой Churn сделаем бинарными.

In [5]:
state_enc = LabelEncoder()
df["State"] = state_enc.fit_transform(df["State"])
df["International plan"] = (df["International plan"] == "Yes").astype("int")
df["Voice mail plan"] = (df["Voice mail plan"] == "Yes").astype("int")
df["Churn"] = (df["Churn"]).astype("int")

Разделим данные на обучающую и тестовую выборки в отношении 8:2. Инициализируем соответственно на объекты DMatrix dtrain и dtest.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Churn", axis=1),
    df["Churn"],
    test_size=0.2,
    stratify=df["Churn"],
    random_state=17,
)
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

Посмотрим на статистику полученных объектов:

In [7]:
print(
    "Train dataset contains {0} rows and {1} columns".format(
        dtrain.num_row(), dtrain.num_col()
    )
)
print(
    "Test dataset contains {0} rows and {1} columns".format(
        dtest.num_row(), dtest.num_col()
    )
)

Train dataset contains 2666 rows and 19 columns
Test dataset contains 667 rows and 19 columns


In [8]:
print("Train mean target: ")
print(np.mean(dtrain.get_label()))

print("\nTest mean target: ")
print(np.mean(dtest.get_label()))

Train mean target: 
0.1447862

Test mean target: 
0.14542729



### Инициализация параметров

- бинарная классификация ('objective':'binary:logistic')
- ограничим глубину деревьев ('max_depth':3)
- не хотим лишнего вывода ('silent':1)
- проведем 50 итераций бустинга
- шаг градиентного спуска довольно большой ('eta':1) - алгоритм будет обучаться быстро и "агрессивно" (лучше результаты будут, если уменьшить eta и увеличить число итераций)



In [9]:
params = {"objective": "binary:logistic", "max_depth": 3, "silent": 1, "eta": 1}

num_rounds = 50

### Обучение классификатора

Тут мы просто передаем слоавть параметров, данные и число итераций.

In [10]:
xgb_model = xgb.train(params, dtrain, num_rounds)

С помощью watchlist отслеживать качество алгоритма на тестовой выборке для каждой итерации.

In [11]:
watchlist = [(dtest, "test"), (dtrain, "train")]  # native interface only
xgb_model = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-error:0.109445	train-error:0.088897
[1]	test-error:0.110945	train-error:0.076519
[2]	test-error:0.071964	train-error:0.049887
[3]	test-error:0.074963	train-error:0.046887
[4]	test-error:0.071964	train-error:0.042386
[5]	test-error:0.070465	train-error:0.04126
[6]	test-error:0.068966	train-error:0.03901
[7]	test-error:0.068966	train-error:0.039385
[8]	test-error:0.070465	train-error:0.03901
[9]	test-error:0.070465	train-error:0.034134
[10]	test-error:0.068966	train-error:0.033383
[11]	test-error:0.071964	train-error:0.032633
[12]	test-error:0.071964	train-error:0.031508
[13]	test-error:0.067466	train-error:0.031133
[14]	test-error:0.073463	train-error:0.028882
[15]	test-error:0.073463	train-error:0.028507
[16]	test-error:0.065967	train-error:0.026257
[17]	test-error:0.064468	train-error:0.025881
[18]	test-error:0.067466	train-error:0.027007
[19]	test-error:0.067466	train-error:0.026257
[20]	test-error:0.068966	train-error:0.025881
[21]	test-error:0.067466	train-error:0.024381
[

Прогнозы для тестовой выборки

In [12]:
preds_prob = xgb_model.predict(dtest)

Посчитаем долю правильных ответов алгоритма на тестовой выборке.

In [13]:
predicted_labels = preds_prob > 0.5
print(
    "Accuracy and F1 on the test set are: {} and {}".format(
        round(accuracy_score(y_test, predicted_labels), 3),
        round(f1_score(y_test, predicted_labels), 3),
    )
)

Accuracy and F1 on the test set are: 0.931 and 0.736


### Задача поиграться с параметрами для нахождение наилучшего результата.

Параметры:

- "num_round": 100,
- "learning_rate": hp.quniform("eta", 0.005, 0.05, 0.005),
- "max_depth": hp.quniform("max_depth", 3, 14, 1),
- "min_child_weight": hp.quniform("min_child_weight", 1, 10, 1),
- "subsample": hp.quniform("subsample", 0.5, 1, 0.05),
- "gamma": hp.quniform("gamma", 0.5, 1, 0.01),
- "colsample_bytree": hp.quniform("colsample_bytree", 0.4, 1, 0.05),
- "num_class": 7,
- "eval_metric": "merror",
- "objective": "multi:softprob",
- "nthread": 4,
- "silent": 1,